![alt text](Python01-basics_extra/logo.png) 

# Итераторы, генераторы, декораторы

Тема непростая, будем слегка жертвовать формализмом и точностью в пользу понятности, а существование некотрых вещей вообще проигнорируем. Те из вас, кто захотят больше деталей и чётких формулировок найдут их в избытке в документации и интернете.

# Итераторы

Давайте вспомним цикл for, который мы прошли во второй лекции (основы питона 2), и еще раз на него посмотрим

In [2]:
for var in range(5):
    print (var)

0
1
2
3
4


In [3]:
for letter in 'Hello world':
    print (letter)

H
e
l
l
o
 
w
o
r
l
d


In [4]:
for key in {'Name':'John', 
            'Age':15, 
            'Gender':'Male', 
           }:
    print (key)

Name
Age
Gender


In [5]:
for line in open('mypackage.py', 'r'):
    print (line)

def myfunc1():

    print('My func 1')

    

def myfunc2(x, y=None):

    if y is not None:

        print(x, y)

    else:

        print(x)

    return



То есть мы что, можем идти циклом по любому объекту? А это вообще законно?

Давайте разбираться, как мы помним из третьей лекции (ООП) - все объекты являются классами. Рассмотрим их методы

In [6]:
dir(range(5))

['__bool__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'count',
 'index',
 'start',
 'step',
 'stop']

In [7]:
dir({'Name':'John', 
            'Age':15, 
            'Gender':'Male', 
           })

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

Метод  *\_\_iter*\_\_  возвращает экземпляр класса, и далее переходит к следующему объекту согласно заданному алгоритму (в тривиальном случае - k+1й объект) посредством метода *\_\_next*\_\_ пока не получит ислючение **StopIteration**

In [8]:
a = [0,1,2,3,4,5,6,7,8,9]
my_iter = iter(a)

In [9]:
print(my_iter.__next__()) # Выглядит жутковато

0


In [10]:
print(next(my_iter)) # Вроде как-то попривычнее

1


In [11]:
next(my_iter) # Будем делать так, чтобы уменьшить синтаксическую нагрузку

2

Метод  *\_\_iter*\_\_ может возвращать self (т.е. себя), то есть итератор от итератора - тоже итератор, причём тот же самый. 

In [12]:
a = [0,1,2,3,4,5]
my_iter = iter(iter(a))

In [13]:
next(my_iter)

0

Если быть достаточно наблюдательным, то можно заметить, что метод *\_\_next*\_\_ отсутствует у наших исходных объектов. Однако он появляется когда мы делаем из него итератор

In [14]:
dir(a)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [15]:
dir(iter(a))

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__length_hint__',
 '__lt__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [16]:
# Как проверить объект на итерируемость
my_object = [1,2,3]
#my_object = 2
#my_object = 'hello'
try:
    iter(my_object)
except TypeError:
    print('not iterable')
else:
    print('iterable')

iterable


Давайте рассмотрим **range**, это даст нам новый взгляд на итераторы. Допустим, нам надо проитерироваться по какому-то очень большому циклу. Втупую, для этого пришлось бы записать всю последовательность элементов в память, создать итератор, и идти от предыдущего к следующему много раз. Но держать в памяти этот огромный объект не слишком-то хотелось бы. Тут нас и выручают итераторы, которые не держат в памяти объект, а знают только своё текущее состояние и закон по которому выбираем следующий элемент.

In [17]:
range(5) # range не существует в каком-то полном запомненном виде

range(0, 5)

In [18]:
a = range(5)
# Но range на самом деле не итератор, это итерируемый объект
next(a)

TypeError: 'range' object is not an iterator

In [19]:
b = iter(range(5)) # А вот это - итератор по range

In [20]:
next(b)

0

In [21]:
# Давайте проверим, а вдруг теперь когда мы сделали из range итератор, там всё же записали весь объект
iter(range(5))

Ну, раз уж мы знаем ООП, давайте напишем свой класс с итератором. Это поможет нам лучше понять на что надо обращать внимание и как вообще их писать 

In [22]:
class NaiveRange(object):
    def __init__(self, start, end):
        self.end = end
        self.ind = start
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.ind == self.end:
            raise StopIteration
        self.ind += 1
        return (self.ind -1)

In [23]:
a = NaiveRange(3,9)

In [24]:
list(a)

[3, 4, 5, 6, 7, 8]

In [25]:
list(a)

[]

Дело в том, что архитектура нашего класса не позволяет индексу вернуться к исходному состоянию без ре-инициализации, он запомнен и с этим ничего не поделать. Такая итерация называется **исчерпаемой**. Пример: итерация по файлу. Мы можем его открыть и по мере прочтения достигнем конца. Конечно есть всякие трюки, но базово мы можем прочитать файл только раз, далее должны открыть его заново (и ре-инициализировать итератор).

Давайте теперь построим неисчерпываемый итератор.

In [26]:
class BaseIter(object):
    def __init__(self, start, end):
        self.end = end
        self.ind = start
        
    def __next__(self):
        if self.ind == self.end:
            raise StopIteration
        self.ind += 1
        return (self.ind -1)
    
    
class AdvancedRange(object):
    def __init__(self, start, end):
        self.end = end
        self.start = start
    def __iter__(self):
        return BaseIter(self.start, self.end)

In [27]:
a = AdvancedRange(3,9)

In [28]:
list(a)

[3, 4, 5, 6, 7, 8]

In [29]:
list(a)

[3, 4, 5, 6, 7, 8]

Теперь, при вызове итератора, мы создаем объект класса BaseIter и можем так делать сколько угодно раз, и каждый из них будет держать в своей памяти своё состояние, не мешая остальным итерироваться. Это, соответственно, **неисчерпаемая** итерация. Пример: список, словарь.

Теперь, с учётом полученных сведений, давайте вернёмся к тому с чего начинали, к циклу *for*

In [30]:
some_iter = iter(some_object)
while True:
    try:
        some_value = next(some_iter)
    except StopIteration:
        break
    do_smth(some_value)

NameError: name 'some_object' is not defined

# Генераторы

Идея следующая. Сделаем такую функцию, которая будет по какому-то входному параметру создавать итераторы. Ключевой является команда **yield**, которая работает примерно как *return* только при следующем обращении проход по телу будет осуществляться не с начала, а с места где мы закончили. Разберём пару примеров и станет ясно зачем это всё надо. 
![alt text](Python06-iterators/iter_vs_gen.svg)

In [31]:
def my_first_genertor(var):
    yield 'first launch'
    yield 'second launch'
    some_result = var*3+2
    yield some_result
    yield 'last launch'
    print ('Done, no more yields, only an exception - StopIteration')

In [32]:
gen = my_first_genertor(2)
#list(gen)

In [33]:
next(gen)

'first launch'

Далее, сколько бы мы не вызывали, если не ре-инициализировать генератор, то он больше ничего не выдаст. Генератор **исчерпался**. 

**Return** же в свою очередь в генераторе не работает как обычно, а работает исключительно как *StopIteration*. Если return не написать, то он подразумевается в конце.

In [34]:
def my_second_genertor(var):
    yield 'first launch'
    return 'second launch' # Заменим yield на return
    some_result = var*3+2
    yield some_result
    yield 'last launch'
    print ('Done, no more yields, only an exception - StopIteration')

In [35]:
gen = my_second_genertor(2)
#list(gen)

In [36]:
next(gen)

'first launch'

Заметим, что запустив повторно мы больше не переходим к следующему yield. Генератор исчерпался по исключению, не зависимо от того есть ли там ещё yield.

Зачем же всё это может нам понадобиться? Давайте вспомним одну классную *штуку* которую мы видели на второй лекции

In [37]:
nums = ['zero', 'one', 'two', 'three', 'four']
for index, number in enumerate(nums):
    print(index, number)

0 zero
1 one
2 two
3 three
4 four


**Enumerate** при итерации по объекту присваивает порядковые номера объектам, очень удобно и рекомендуется к использованию. *Enumerate* ни что иное как генератор, и может быть записано следующим образом

In [38]:
def my_enumerate(iterable_object):
    ind = 0
    for smth in iterable_object:
        yield ind, smth
        ind += 1

In [39]:
for index, number in my_enumerate(nums):
    print(index, number)

0 zero
1 one
2 two
3 three
4 four


Когда встречаешь что-то такое первый раз, то невольно возникает мысль "Ну написали цикл в функции, и зачем это всё? Я и сам могу цикл написать и безо всяких генераторов". Однако, стоит обратить внимание на то, что генератор не держит в памяти полностью записанный объект, вместо этого просто обращается к методу *next*, и это очень важно.

In [40]:
print(my_enumerate(nums))

<generator object my_enumerate at 0x7fdfdc31dbd0>


Давайте рассмотрим ещё несколько генераторов

In [41]:
# map(func, smth_iterable) применяет к каждом элементу функцию func
double = map(lambda x: x + x, nums) 
print(list(double))

['zerozero', 'oneone', 'twotwo', 'threethree', 'fourfour']


In [42]:
# Потренируемся, сделаем свой map
def my_map(func, iterable_object):
    for smth in iterable_object:
        yield func(smth)

In [43]:
my_double = my_map(lambda x: x + x, nums) 
print(list(my_double))

['zerozero', 'oneone', 'twotwo', 'threethree', 'fourfour']


In [44]:
# filter(cond, smth_iterable) применяет к каждому элементу проверку по условию cond и отфильтровывает
len_filter = filter(lambda x: len(x) > 3, nums)
print(list(len_filter))

['zero', 'three', 'four']


In [45]:
# Cделаем свой filter
def my_filter(condition, iterable_object):
    for smth in iterable_object:
        if condition(smth):
            yield smth

In [46]:
# filter(cond, smth_iterable) применяет к каждому элементу проверку по условию cond и отфильтровывает
my_len_filter = my_filter(lambda x: len(x) > 3, nums)
print(list(my_len_filter))

['zero', 'three', 'four']


Есть множество полезных генераторов, которые пока ещё не впилили в основной питон (но судя по логам версий - этим занимаются): count, repeat, cycle, chain и другие. Их можно найти в пакете itertools. Давайте посмотрим.

In [47]:
from itertools import repeat
# Повторяет то что ему дали

In [48]:
for var in repeat(5):
    print(var)

5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


...

5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


KeyboardInterrupt: 

In [49]:
from itertools import chain
# "Раскрывает" итерируемые объекты и делает из них один. Но делает это только на 1 уровень в "глубину"

In [50]:
list(chain('A', 'B', [1, 2, 3, None], 'a string', range(4,7)))

['A', 'B', 1, 2, 3, None, 'a', ' ', 's', 't', 'r', 'i', 'n', 'g', 4, 5, 6]

In [51]:
list(chain('A', 'B', [1, 2, 3, 'a string', None], [range(4,7)]))

['A', 'B', 1, 2, 3, 'a string', None, range(4, 7)]

In [52]:
from itertools import count
# Принимает начальное состояние и шаг и делает с ними бесконечный цикл

In [53]:
for i in count(0,3):
    print(i)

0
3
6
9
12
15
18
21
24
27
30
33
36
39
42
45
48
51
54
57
60
63
66
69
72
75
78
81
84
87
90
93
96
99
102
105
108
111
114
117
120
123
126
129
132
135
138
141
144
147
150
153
156
159
162
165
168
171
174
177
180
183
186
189
192
195
198
201
204
207
210
213
216
219
222
225
228
231
234
237
240
243
246
249
252
255
258
261
264
267
270
273
276
279
282
285
288
291
294
297
300
303
306
309
312
315
318
321
324
327
330
333
336
339
342
345
348
351
354
357
360
363
366
369
372
375
378
381
384
387
390
393
396
399
402
405
408
411
414
417
420
423
426
429
432
435
438
441
444
447
450
453
456
459
462
465
468
471
474
477
480
483
486
489
492
495
498
501
504
507
510
513
516
519
522
525
528
531
534
537
540
543
546
549
552
555
558
561
564
567
570
573
576
579
582
585
588
591
594
597
600
603
606
609
612
615
618
621
624
627
630
633
636
639
642
645
648
651
654
657
660
663
666
669
672
675
678
681
684
687
690
693
696
699
702
705
708
711
714
717
720
723
726
729
732
735
738
741
744
747
750
753
756
759
762
765
768
771
774
77

9015
9018
9021
9024
9027
9030
9033
9036
9039
9042
9045
9048
9051
9054
9057
9060
9063
9066
9069
9072
9075
9078
9081
9084
9087
9090
9093
9096
9099
9102
9105
9108
9111
9114
9117
9120
9123
9126
9129
9132
9135
9138
9141
9144
9147
9150
9153
9156
9159
9162
9165
9168
9171
9174
9177
9180
9183
9186
9189
9192
9195
9198
9201
9204
9207
9210
9213
9216
9219
9222
9225
9228
9231
9234
9237
9240
9243
9246
9249
9252
9255
9258
9261
9264
9267
9270
9273
9276
9279
9282
9285
9288
9291
9294
9297
9300
9303
9306
9309
9312
9315
9318
9321
9324
9327
9330
9333
9336
9339
9342
9345
9348
9351
9354
9357
9360
9363
9366
9369
9372
9375
9378
9381
9384
9387
9390
9393
9396
9399
9402
9405
9408
9411
9414
9417
9420
9423
9426
9429
9432
9435
9438
9441
9444
9447
9450
9453
9456
9459
9462
9465
9468
9471
9474
9477
9480
9483
9486
9489
9492
9495
9498
9501
9504
9507
9510
9513
9516
9519
9522
9525
9528
9531
9534
9537
9540
9543
9546
9549
9552
9555
9558
9561
9564
9567
9570
9573
9576
9579
9582
9585
9588
9591
9594
9597
9600
9603
9606
9609
9612


16995
16998
17001
17004
17007
17010
17013
17016
17019
17022
17025
17028
17031
17034
17037
17040
17043
17046
17049
17052
17055
17058
17061
17064
17067
17070
17073
17076
17079
17082
17085
17088
17091
17094
17097
17100
17103
17106
17109
17112
17115
17118
17121
17124
17127
17130
17133
17136
17139
17142
17145
17148
17151
17154
17157
17160
17163
17166
17169
17172
17175
17178
17181
17184
17187
17190
17193
17196
17199
17202
17205
17208
17211
17214
17217
17220
17223
17226
17229
17232
17235
17238
17241
17244
17247
17250
17253
17256
17259
17262
17265
17268
17271
17274
17277
17280
17283
17286
17289
17292
17295
17298
17301
17304
17307
17310
17313
17316
17319
17322
17325
17328
17331
17334
17337
17340
17343
17346
17349
17352
17355
17358
17361
17364
17367
17370
17373
17376
17379
17382
17385
17388
17391
17394
17397
17400
17403
17406
17409
17412
17415
17418
17421
17424
17427
17430
17433
17436
17439
17442
17445
17448
17451
17454
17457
17460
17463
17466
17469
17472
17475
17478
17481
17484
17487
17490
1749

25746
25749
25752
25755
25758
25761
25764
25767
25770
25773
25776
25779
25782
25785
25788
25791
25794
25797
25800
25803
25806
25809
25812
25815
25818
25821
25824
25827
25830
25833
25836
25839
25842
25845
25848
25851
25854
25857
25860
25863
25866
25869
25872
25875
25878
25881
25884
25887
25890
25893
25896
25899
25902
25905
25908
25911
25914
25917
25920
25923
25926
25929
25932
25935
25938
25941
25944
25947
25950
25953
25956
25959
25962
25965
25968
25971
25974
25977
25980
25983
25986
25989
25992
25995
25998
26001
26004
26007
26010
26013
26016
26019
26022
26025
26028
26031
26034
26037
26040
26043
26046
26049
26052
26055
26058
26061
26064
26067
26070
26073
26076
26079
26082
26085
26088
26091
26094
26097
26100
26103
26106
26109
26112
26115
26118
26121
26124
26127
26130
26133
26136
26139
26142
26145
26148
26151
26154
26157
26160
26163
26166
26169
26172
26175
26178
26181
26184
26187
26190
26193
26196
26199
26202
26205
26208
26211
26214
26217
26220
26223
26226
26229
26232
26235
26238
26241
2624

34455
34458
34461
34464
34467
34470
34473
34476
34479
34482
34485
34488
34491
34494
34497
34500
34503
34506
34509
34512
34515
34518
34521
34524
34527
34530
34533
34536
34539
34542
34545
34548
34551
34554
34557
34560
34563
34566
34569
34572
34575
34578
34581
34584
34587
34590
34593
34596
34599
34602
34605
34608
34611
34614
34617
34620
34623
34626
34629
34632
34635
34638
34641
34644
34647
34650
34653
34656
34659
34662
34665
34668
34671
34674
34677
34680
34683
34686
34689
34692
34695
34698
34701
34704
34707
34710
34713
34716
34719
34722
34725
34728
34731
34734
34737
34740
34743
34746
34749
34752
34755
34758
34761
34764
34767
34770
34773
34776
34779
34782
34785
34788
34791
34794
34797
34800
34803
34806
34809
34812
34815
34818
34821
34824
34827
34830
34833
34836
34839
34842
34845
34848
34851
34854
34857
34860
34863
34866
34869
34872
34875
34878
34881
34884
34887
34890
34893
34896
34899
34902
34905
34908
34911
34914
34917
34920
34923
34926
34929
34932
34935
34938
34941
34944
34947
34950
3495

...

59163
59166
59169
59172
59175
59178
59181
59184
59187
59190
59193
59196
59199
59202
59205
59208
59211
59214
59217
59220
59223
59226
59229
59232
59235
59238
59241
59244
59247
59250
59253
59256
59259
59262
59265
59268
59271
59274
59277
59280
59283
59286
59289
59292
59295
59298
59301
59304
59307
59310
59313
59316
59319
59322
59325
59328
59331
59334
59337
59340
59343
59346
59349
59352
59355
59358
59361
59364
59367
59370
59373
59376
59379
59382
59385
59388
59391
59394
59397
59400
59403
59406
59409
59412
59415
59418
59421
59424
59427
59430
59433
59436
59439
59442
59445
59448
59451
59454
59457
59460
59463
59466
59469
59472
59475
59478
59481
59484
59487
59490
59493
59496
59499
59502
59505
59508
59511
59514
59517
59520
59523
59526
59529
59532
59535
59538
59541
59544
59547
59550
59553
59556
59559
59562
59565
59568
59571
59574
59577
59580
59583
59586
59589
59592
59595
59598
59601
59604
59607
59610
59613
59616
59619
59622
59625
59628
59631
59634
59637
59640
59643
59646
59649
59652
59655
59658
5966

66780
66783
66786
66789
66792
66795
66798
66801
66804
66807
66810
66813
66816
66819
66822
66825
66828
66831
66834
66837
66840
66843
66846
66849
66852
66855
66858
66861
66864
66867
66870
66873
66876
66879
66882
66885
66888
66891
66894
66897
66900
66903
66906
66909
66912
66915
66918
66921
66924
66927
66930
66933
66936
66939
66942
66945
66948
66951
66954
66957
66960
66963
66966
66969
66972
66975
66978
66981
66984
66987
66990
66993
66996
66999
67002
67005
67008
67011
67014
67017
67020
67023
67026
67029
67032
67035
67038
67041
67044
67047
67050
67053
67056
67059
67062
67065
67068
67071
67074
67077
67080
67083
67086
67089
67092
67095
67098
67101
67104
67107
67110
67113
67116
67119
67122
67125
67128
67131
67134
67137
67140
67143
67146
67149
67152
67155
67158
67161
67164
67167
67170
67173
67176
67179
67182
67185
67188
67191
67194
67197
67200
67203
67206
67209
67212
67215
67218
67221
67224
67227
67230
67233
67236
67239
67242
67245
67248
67251
67254
67257
67260
67263
67266
67269
67272
67275
6727

74700
74703
74706
74709
74712
74715
74718
74721
74724
74727
74730
74733
74736
74739
74742
74745
74748
74751
74754
74757
74760
74763
74766
74769
74772
74775
74778
74781
74784
74787
74790
74793
74796
74799
74802
74805
74808
74811
74814
74817
74820
74823
74826
74829
74832
74835
74838
74841
74844
74847
74850
74853
74856
74859
74862
74865
74868
74871
74874
74877
74880
74883
74886
74889
74892
74895
74898
74901
74904
74907
74910
74913
74916
74919
74922
74925
74928
74931
74934
74937
74940
74943
74946
74949
74952
74955
74958
74961
74964
74967
74970
74973
74976
74979
74982
74985
74988
74991
74994
74997
75000
75003
75006
75009
75012
75015
75018
75021
75024
75027
75030
75033
75036
75039
75042
75045
75048
75051
75054
75057
75060
75063
75066
75069
75072
75075
75078
75081
75084
75087
75090
75093
75096
75099
75102
75105
75108
75111
75114
75117
75120
75123
75126
75129
75132
75135
75138
75141
75144
75147
75150
75153
75156
75159
75162
75165
75168
75171
75174
75177
75180
75183
75186
75189
75192
75195
7519

83448
83451
83454
83457
83460
83463
83466
83469
83472
83475
83478
83481
83484
83487
83490
83493
83496
83499
83502
83505
83508
83511
83514
83517
83520
83523
83526
83529
83532
83535
83538
83541
83544
83547
83550
83553
83556
83559
83562
83565
83568
83571
83574
83577
83580
83583
83586
83589
83592
83595
83598
83601
83604
83607
83610
83613
83616
83619
83622
83625
83628
83631
83634
83637
83640
83643
83646
83649
83652
83655
83658
83661
83664
83667
83670
83673
83676
83679
83682
83685
83688
83691
83694
83697
83700
83703
83706
83709
83712
83715
83718
83721
83724
83727
83730
83733
83736
83739
83742
83745
83748
83751
83754
83757
83760
83763
83766
83769
83772
83775
83778
83781
83784
83787
83790
83793
83796
83799
83802
83805
83808
83811
83814
83817
83820
83823
83826
83829
83832
83835
83838
83841
83844
83847
83850
83853
83856
83859
83862
83865
83868
83871
83874
83877
83880
83883
83886
83889
83892
83895
83898
83901
83904
83907
83910
83913
83916
83919
83922
83925
83928
83931
83934
83937
83940
83943
8394

KeyboardInterrupt: 

### Выражения-генераторы

Стандарт Pep-289 даёт нам подробные инструкции как надо записывать и вообще как пользоваться выражениями-генераторами https://www.python.org/dev/peps/pep-0289/ 
Например,

In [54]:
def gen(exp):
    for x in exp:
        yield x**2
g = gen(iter(range(10)))

# Эквивалентно
g = (x**2 for x in range(10))

Это удобно краткой записью в одну строку, при этом это не просто выражение, а генератор, со всеми вытекающими преимуществами и свойствами, которые мы ранее изучили. 

![alt text](Python06-iterators/deeper.jpg) 

Давайте теперь рассмотрим такой пример, когда у нас есть два генератора, которые общаются между собой 

In [56]:
import random

# Генерим новые данные
def get_data():
    return random.sample(range(10), 2)

# Обрабатываем данные
def consume():
    total_sum = 0
    total_amount = 0
    
    while True:
        #data = yield
        data = yield
        total_sum += sum(data)
        total_amount += len(data)
        print('Running average is {}'.format(total_sum/total_amount))
        
# Производим новые данные и передаём в обработку
def produce(consumer):
    while True:
        data = get_data()
        print('New data {}'.format(data))
        consumer.send(data)
        yield

In [57]:
consumer = consume()
next(consumer)
producer = produce(consumer)

for _ in range(1000):
    next(producer)

New data [2, 3]
Running average is 2.5
New data [4, 9]
Running average is 4.5
New data [0, 9]
Running average is 4.5
New data [5, 7]
Running average is 4.875
New data [9, 2]
Running average is 5.0
New data [0, 3]
Running average is 4.416666666666667
New data [7, 6]
Running average is 4.714285714285714
New data [2, 0]
Running average is 4.25
New data [5, 2]
Running average is 4.166666666666667
New data [9, 4]
Running average is 4.4
New data [8, 9]
Running average is 4.7727272727272725
New data [3, 9]
Running average is 4.875
New data [8, 0]
Running average is 4.8076923076923075
New data [4, 2]
Running average is 4.678571428571429
New data [3, 8]
Running average is 4.733333333333333
New data [8, 2]
Running average is 4.75
New data [0, 4]
Running average is 4.588235294117647
New data [5, 7]
Running average is 4.666666666666667
New data [5, 7]
Running average is 4.7368421052631575
New data [3, 9]
Running average is 4.8
New data [4, 0]
Running average is 4.666666666666667
New data [1, 8]
Ru

Чем хороша приведённая конструкция (её идея)? Мы не просто разделили процесс генерации и обработки данных, это можно было сделать и без всяких генераторов. **Send** создаёт отдельный thread вычислений, которые производятся параллельно основному телу программы. Таким образом, генерить и обрабатывать данные многими разными способами можно в параллели, притормаживая только по ключевому слову *yield* чтобы дождаться данных или следующего обращения. 

Теперь, можем сказать что для генераторов next(a) эквивалентент a.send(None).

Всё это называется Coroutine и поподробнее можно почитать в PEP-342 https://www.python.org/dev/peps/pep-0342/

![alt text](Python06-iterators/coroutine.jpg)

Эта идея позволяетнам, например, создать планировщик задач, который по определенным условиям будет запускать тот или иной генератор, и экономить время выполняя их в параллели и переключаясь на них только когда это требуется. 

*Те кто переживает что плохо понял тему, и кто хочет разобраться и закрепить в памяти материал по генераторам, могут после лекции попробовать написать генератор, который в параллели качнет наши прошедшие лекции, обложить таймерами, и убедиться что это действительно работает быстрее. Это не обязательное задание, но это поможет усвоить материал. Ничего никуда посылать не надо*

Последнее, что хотелось бы рассмотреть это конструкция **yield from**. Обычно говорят, что эта конструкция устанавливает двухнаправленную связь между вызывающим генератором и суб-генератором, передавая, в том числе, исключения (transparent two way channel).

Сначала простоьй пример:

In [59]:
def generator_simple():
    for i in range(10):
        yield i
    for j in range(10, 20):
        yield j

Он, будет выдавать нам числа от 0 до 19. И мы решили разбить его на две части чтобы их можно было еще использовать по отдельности.

In [60]:
def generator1():
    for i in range(10):
        yield i

def generator2():
    for j in range(10, 20):
        yield j
        
def generator():
    for i in generator1():
        yield i
    for j in generator2():
        yield j

In [61]:
# То же самое можно записать в виде
def generator_from():
    yield from generator1()
    yield from generator2()

Понятно, что это не очень впечатляет, так что давайте посмотрим чуть более сложный пример

In [62]:
def writer():
    """A coroutine that writes data *sent* to it """
    while True:
        w = (yield)
        print('>> ', w)

def writer_wrapper(coro):
    coro.send(None)  # Для инициализации
    while True:
        try:
            x = (yield)  # Принимаем
            coro.send(x)  # и передаём
        except StopIteration:
            pass

In [63]:
w = writer()
wrap = writer_wrapper(w)
wrap.send(None)
for i in range(4):
    wrap.send(i)

>>  0
>>  1
>>  2
>>  3


In [64]:
#Всё то же самое выполняется при помощи yield from
def writer_wrapper_from(coro):
    yield from coro

In [65]:
w = writer()
wrap = writer_wrapper_from(w)
wrap.send(None)
for i in range(4):
    wrap.send(i)

>>  0
>>  1
>>  2
>>  3


Если подытожить, то yield from позволяет серьезно упростить код и не думать о всяких вещах, о которых мы обычно не хотим думать, и **не теряет** при этом никаких важных исключений или типа того.

# Декораторы

Допустим, мы столкнулись с задачей, что нам надо подправить кое-какие функции, но лезть в них и менять код прямо там нам не сильно охота, потому что, например, мы можем потом передумать. Или переживаем за общую совместимость в большом проекте. Сделаем обёртку нужной нам функции:

In [66]:
def deprecated(func):
    def wrap(*args, **kwargs):
        print ('{} will be deprecated soon'.format(func.__name__))
        return func(*args, **kwargs)
    return wrap

In [67]:
def my_func(x):
    return x+3

# "Обернём" нашу функцию посредством переприсвоения
my_func = deprecated(my_func)

In [68]:
my_func(2)

my_func will be deprecated soon


5

In [69]:
# Ровно то же самое можно записать без переприсвоения:
@deprecated # Это декоратор
def my_func(x):
    return x+3

In [70]:
my_func(2)

my_func will be deprecated soon


5

In [71]:
# Но, заметим, что именем ф-ии стало имя обёртки
my_func.__name__

'wrap'

Чтобы сделать красивый декоратор и совсем ничего в функции не менять, кроме того что мы поменять хотим, давайте воспользуемся библиотекой functools

In [72]:
import functools

def deprecated(func):
    @functools.wraps(func)
    def wrap(*args, **kwargs):
        print ('{} will be deprecated soon'.format(func.__name__))
        return func(*args, **kwargs)
    return wrap

@deprecated
def my_func(x):
    return x+3

my_func.__name__

'my_func'

Декораторы работают с классами так же как и с функциями. Создадим декоратор выводящий время работы декорируемой функции, простой класс производящий какую-то операцию, наложим и посмотрим

In [73]:
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer


In [74]:
class TimeWaster:    
    def __init__(self, max_num):
        self.max_num = max_num

    #@timer
    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])        

In [75]:
tw = TimeWaster(1000)
tw.waste_time(999)

Допустим, мы хотим чтобы наш декоратор что-то нам важное вывел, например исключение. Только не в общий вывод, чтобы не засорять, а куда-то в другое место, где мы при необходимости прочитаем. Для этого можно использовать атрибуты декоратора

In [76]:
def redirected_output(destination=None):        
    def wrapper(*args, **kwargs):
        with open(destination, 'w') as output:
            print('Some important information', 
                  file=output)    
    return wrapper

In [77]:
@redirected_output(destination='log.txt')
def foo():
    pass

Этот пример не очень хорош, зато он прост и демонстрирует что в сам декоратор можно передавать входные данные